# Entrenamiento de CNN para Clasificación de Enfermedades en Plantas

Este notebook implementa una red neuronal convolucional (CNN) para clasificar enfermedades en hojas de plantas usando el dataset PlantVillage.


In [ ]:
# Importaciones necesarias
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import os


## 1. Definición de rutas y constantes


In [ ]:
# Rutas y constantes
DATASET_PATH = Path('../dataset/raw/plant_diseases')
MODEL_PATH = Path('../models/modelo_hojas.h5')

# Parámetros de la imagen (según RF-02)
IMG_SIZE = (224, 224)  # target_size
COLOR_MODE = 'rgb'  # color_mode
BATCH_SIZE = 32
EPOCHS = 10  # Cantidad pequeña para demostración

# División del dataset
TRAIN_SPLIT = 0.8  # 80% entrenamiento, 20% validación

print(f"📁 Dataset: {DATASET_PATH}")
print(f"💾 Modelo se guardará en: {MODEL_PATH}")
print(f"🖼️  Tamaño de imagen: {IMG_SIZE}")
print(f"🎨 Modo de color: {COLOR_MODE}")


## 2. Generación de Data Generators con ImageDataGenerator


In [ ]:
# Verificar que el dataset existe
if not DATASET_PATH.exists():
    raise FileNotFoundError(f"El dataset no se encuentra en {DATASET_PATH}")

# Data Generator para entrenamiento con aumento de datos
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalización (valores entre 0 y 1)
    rotation_range=20,  # Rotación aleatoria hasta 20 grados
    width_shift_range=0.2,  # Desplazamiento horizontal
    height_shift_range=0.2,  # Desplazamiento vertical
    zoom_range=0.2,  # Zoom aleatorio
    horizontal_flip=True,  # Volteo horizontal
    validation_split=TRAIN_SPLIT  # División entrenamiento/validación
)

# Data Generator para validación (solo normalización, sin aumento)
val_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalización
    validation_split=TRAIN_SPLIT
)

print("✅ Data Generators configurados")


In [ ]:
# Generadores de datos para entrenamiento y validación
train_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,  # Redimensionar a 224x224
    color_mode=COLOR_MODE,  # RGB
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # Para clasificación multiclase
    subset='training',  # Subconjunto de entrenamiento
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    color_mode=COLOR_MODE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',  # Subconjunto de validación
    shuffle=False
)

# Obtener número de clases
NUM_CLASSES = len(train_generator.class_indices)
CLASS_NAMES = list(train_generator.class_indices.keys())

print(f"✅ Generadores creados")
print(f"📊 Número de clases: {NUM_CLASSES}")
print(f"📈 Imágenes de entrenamiento: {train_generator.samples}")
print(f"📉 Imágenes de validación: {val_generator.samples}")
print(f"\n📋 Primeras 10 clases: {CLASS_NAMES[:10]}")


## 3. Definición de la Arquitectura CNN


In [ ]:
# Crear modelo Sequential
model = Sequential([
    # Primer bloque convolucional
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D(2, 2),
    
    # Segundo bloque convolucional
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    # Tercer bloque convolucional (opcional, para mejorar rendimiento)
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    # Aplanar las características
    Flatten(),
    
    # Primera capa densa
    Dense(512, activation='relu'),
    Dropout(0.5),  # Regularización para evitar overfitting
    
    # Segunda capa densa
    Dense(256, activation='relu'),
    Dropout(0.5),
    
    # Capa de salida con Softmax para clasificación multiclase
    Dense(NUM_CLASSES, activation='softmax')
])

print("✅ Modelo CNN creado")


## 4. Visualización del Resumen del Modelo


In [ ]:
model.summary()

# 5. Compilación del Modelo


In [ ]:
# Compilar el modelo
model.compile(
    optimizer=Adam(learning_rate=0.001),  # Optimizador adam
    loss='categorical_crossentropy',  # Función de pérdida para clasificación multiclase
    metrics=['accuracy']  # Métrica de precisión
)

print("✅ Modelo compilado")
print(f"   Optimizador: Adam")
print(f"   Función de pérdida: categorical_crossentropy")
print(f"   Métrica: accuracy")


## 6. Entrenamiento del Modelo


In [ ]:
# Calcular steps por época
steps_per_epoch = train_generator.samples // BATCH_SIZE
validation_steps = val_generator.samples // BATCH_SIZE

print(f"🔄 Iniciando entrenamiento...")
print(f"   Épocas: {EPOCHS}")
print(f"   Steps por época: {steps_per_epoch}")
print(f"   Validation steps: {validation_steps}")

# Entrenar el modelo
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=validation_steps,
    verbose=1
)

print("✅ Entrenamiento completado")


## 7. Visualización de Resultados del Entrenamiento


In [ ]:
# Graficar accuracy y loss
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Accuracy
axes[0].plot(history.history['accuracy'], label='Train Accuracy')
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy')
axes[0].set_title('Model Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True)

# Loss
axes[1].plot(history.history['loss'], label='Train Loss')
axes[1].plot(history.history['val_loss'], label='Validation Loss')
axes[1].set_title('Model Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

# Mostrar métricas finales
print(f"\n📊 Métricas finales:")
print(f"   Train Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"   Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")
print(f"   Train Loss: {history.history['loss'][-1]:.4f}")
print(f"   Validation Loss: {history.history['val_loss'][-1]:.4f}")


In [ ]:
# Asegurar que el directorio models existe
MODEL_PATH.parent.mkdir(parents=True, exist_ok=True)

# Guardar el modelo
model.save(MODEL_PATH)

print(f"✅ Modelo guardado en: {MODEL_PATH}")
print(f"📦 Tamaño del archivo: {MODEL_PATH.stat().st_size / (1024*1024):.2f} MB")

# Guardar también los nombres de las clases para uso futuro
import json
class_indices_path = MODEL_PATH.parent / 'class_indices.json'
with open(class_indices_path, 'w') as f:
    json.dump(train_generator.class_indices, f, indent=2)

print(f"✅ Índices de clases guardados en: {class_indices_path}")


## 9. Resumen y Próximos Pasos


In [ ]:
print("✅ Entrenamiento completado exitosamente")
print("\n📝 Próximos pasos:")
print("  1. El modelo está guardado en models/modelo_hojas.h5")
print("  2. Los índices de clases están en models/class_indices.json")
print("  3. Integrar el modelo en el backend (app/utils.py y backend/main.py)")
print("  4. Probar el endpoint /predict desde el frontend")
print("\n💡 Nota: Si el accuracy es bajo, considera:")
print("  - Aumentar el número de épocas")
print("  - Ajustar el learning rate")
print("  - Agregar más capas convolucionales")
print("  - Usar transfer learning (ResNet, VGG, etc.)")
